# Census Data

The API documentation is [here](https://www.census.gov/data/developers/data-sets/economic-indicators.html). The link for economic series is in this [link](https://api.census.gov/data/timeseries/eits.html), and the parameters for the requests are listed [here](https://api.census.gov/data/timeseries/eits/m3/variables.html). Finally this [link](https://www.census.gov/econ/currentdata/dbsearch?program=M3ADV&startYear=1992&endYear=2022&categories=MDM&dataType=NO&geoLevel=US&adjusted=1&notAdjusted=0&errorData=0) has the whole data for each survey, you will need to download it once in order to get the attributes for the parameters of the API (I wasn't able to find them elsewhere)

In [1]:
import pandas as pd
import requests
import os

# path to the folder "project"
path = "C:\\Users\\pedro\\OneDrive\\NYU\\CSS\\II. Data Skills\\project"
os.chdir(path)

import keys # file with api keys
key = keys.CENSUS # you can just paste your own key as a string.

## API Structure

For the Economic Indicators, we have the following structure:

### Base Path

- `https://api.census.gov/data/timeseries/eits/`

### Data Set Name

- `code`: The name for each survey

### Variables

Variables don't need to have a value assigned to them, but they need to be stated after the `?Get=`. If any value is assign, returns all possible values.

**Header Variables:** These parameters are in the URL, but don't have values associated with them (separated by `,`):

- `time_slot_id`: date values (nothing to specify)

- `cell_value`: values data (nothing to specify)

**Filter Variables:** Variables that helps to narrow the request (separated by `&`). 

- `data_type_code`: item type code for elements of  each survey. (values are not required).

- `category_code`: Industry code list for each survey (values are not required).

- `seasonally_adj`: possible values are yes or no (values are not required).

- `for`: Refers to the GEO area covered by survey (values are not required).

**Required Variables:**

- `time`: Period of the request. Accept different entries.

- `API Key`: Individual API Key
    

**Example**

All categories from the retail sales seasonally adjusted data for the US in 2021.

- `https://api.census.gov/data/timeseries/eits/marts?get=cell_value,time_slot_id,category_code&seasonally_adj&data_type_code&for=US&time=2021&APIKEY`




## Surveys

Here I listed the possible surveys, and thir code for the API:

### Data Set Code

- Construction Spending (VIP) `vip`
- New Residential Sales (NRS) `ressales`
- New Residential Construction (NRC) `resconst`
- Monthly Wholesale Trade Inventories (MWTS) `mwts`
- Advance Monthly Retail Sales (MARTS) `marts`
- Monthly Trade & Inventory Sales (MTIS) `mtis`
- Monthly Retail Sales (MRTS) `mrts`
- Advance Report on Durable Goods (ADVM3) `advm3`
- Manufacturer’s Shipments Inventories & Orders (M3) `m3`
- International Trade (FTD) `ftd`
- Housing Vacancies & Homeownership Rate (HV) `hv`
- Quarterly Services (QSS) `qss`
- Quarterly Financial Report (QFR) `qfr`
- Business Formation Statistics (BFS) `bfs`


## Advance Report on Durable Goods (ADVM3)

### Category Code

- `MDM` Durable Goods
- `DXT` Durable goods excluding transportation
- `DXD` Durable goods excluding defense
- `MTU` Manufacturing with unfilled orders
- `31S` Primary Metals
- `32S` Fabricated Metal Products
- `33S` Machinery
- `34S` Computers and Electronic Products
- `CRP` Computers and Related products
- `34X` Communications Equipment
- `35S` Electrical Equipment, Appliances and Components
- `36S` Transportation Equipment
- `MVP` Motor Vehicles and Parts
- `NAP` Nondefense Aircraft and Parts
- `DAP` Defense Aircraft and Parts
- `ODG` All Other Durable goods
- `TCG` Capital Goods
- `NDE` Nondefense Capital Goods
- `NXA` Nondefense Capital Goods Excluding Aircraft
- `DEF` Defense Capital Goods


### Data Type

- `VS` Value of Shipments
- `NO` New Orders
- `UO` Unfilled Orders
- `TI` Total Inventories
- `MPCVS` Value of Shipments Percent Change Monthly
- `MPCNO` New Orders Percent Change Monthly
- `MPCUO` Unfilled Orders Percent Change Monthly
- `MPCTI` Total Inventories Percent Change Monthly


In [17]:
indicator = 'advm3'
param = 'cell_value,time_slot_id'
cat = '&category_code' # all categories
datatype = '&data_type_code' # all data types
time = '&time=from+1992' # since start
geo = '&for=us'
adj = '&seasonally_adj=yes'


url = f'https://api.census.gov/data/timeseries/eits/{indicator}?get={param}{cat}{datatype}{geo}{adj}{time}&key={key}'

In [18]:
r = requests.get(url).json()
print(r[0])
print(r[1])

['cell_value', 'time_slot_id', 'category_code', 'data_type_code', 'seasonally_adj', 'time', 'us']
['5101', '633', 'CRP', 'UO', 'yes', '2002-09', '1']


In [19]:
advm = pd.DataFrame(data=r[1:], columns = r[0])
advm.info()
advm.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59080 entries, 0 to 59079
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cell_value      59080 non-null  object
 1   time_slot_id    59080 non-null  object
 2   category_code   59080 non-null  object
 3   data_type_code  59080 non-null  object
 4   seasonally_adj  59080 non-null  object
 5   time            59080 non-null  object
 6   us              59080 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us
0,5101,633,CRP,UO,yes,2002-09,1
1,5090,634,CRP,UO,yes,2002-10,1
2,5325,635,CRP,UO,yes,2002-11,1
3,5774,636,CRP,UO,yes,2002-12,1
4,5508,637,CRP,UO,yes,2003-01,1


In [20]:
# checking values:
advm["time_slot_id"].unique()
advm["category_code"].unique()
advm["data_type_code"].unique()

array(['UO', 'TI', 'NO', 'VS', 'MPCNO', 'MPCVS', 'MPCTI', 'MPCUO'],
      dtype=object)

In [21]:
# changing columns dtypes:
advm["time"]= pd.to_datetime(advm["time"]+"-01")
advm["cell_value"]= pd.to_numeric(advm["cell_value"].str.replace(",", "", regex=False))

# creating series id
advm["series_id"] = "ADVM3" + advm["data_type_code"] + advm["category_code"] + "SAUS"

In [22]:
advm.head()

,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us,series_id
0,5101.0,633,CRP,UO,yes,2002-09-01,1,ADVM3UOCRPSAUS
1,5090.0,634,CRP,UO,yes,2002-10-01,1,ADVM3UOCRPSAUS
2,5325.0,635,CRP,UO,yes,2002-11-01,1,ADVM3UOCRPSAUS
3,5774.0,636,CRP,UO,yes,2002-12-01,1,ADVM3UOCRPSAUS
4,5508.0,637,CRP,UO,yes,2003-01-01,1,ADVM3UOCRPSAUS


In [23]:
# creating mapping dict to data type and category code
category_dict = pd.DataFrame(
    [["MDM","Durable Goods"],["DXT","Durable goods excluding transportation"],
     ["DXD","Durable goods excluding defense"],["MTU","Manufacturing with unfilled orders"],
     ["31S","Primary Metals"],["32S","Fabricated Metal Products"],["33S","Machinery"],
     ["34S","Computers and Electronic Products"],["CRP","Computers and Related products"],
     ["34X","Communications Equipment"],["35S","Electrical Equipment, Appliances and Components"],
     ["36S","Transportation Equipment"],["MVP","Motor Vehicles and Parts"],
     ["NAP","Nondefense Aircraft and Parts"],["DAP","Defense Aircraft and Parts"],
     ["ODG","All Other Durable goods"],["TCG","Capital Goods"],
     ["NDE","Nondefense Capital Goods"],["NXA","Nondefense Capital Goods Excluding Aircraft"],
     ["DEF","Defense Capital Goods"]], columns = ["category_code","category_name"])

data_type_dict = pd.DataFrame(
    [["VS","Value of Shipments"],["NO","New Orders"],["UO","Unfilled Orders"],["TI","Total Inventories"],
     ["MPCVS","Value of Shipments Percent Change Monthly"],["MPCNO","New Orders Percent Change Monthly"],
     ["MPCUO","Unfilled Orders Percent Change Monthly"], ["MPCTI","Total Inventories Percent Change Monthly"]],
    columns = ["data_type_code","data_type_name"])

# merging with data:

advm = pd.merge(advm,data_type_dict,how="left",on="data_type_code")
advm = pd.merge(advm,category_dict,how="left",on="category_code")

In [24]:
# dropping and renaming columns:
advm = (
    advm[["series_id","time","cell_value","category_name","data_type_name"]]
    .rename(columns = {"cell_value":"value", "time":"date"}))

advm.head()

,series_id,date,value,category_name,data_type_name
0,ADVM3UOCRPSAUS,2002-09-01,5101.0,Computers and Related products,Unfilled Orders
1,ADVM3UOCRPSAUS,2002-10-01,5090.0,Computers and Related products,Unfilled Orders
2,ADVM3UOCRPSAUS,2002-11-01,5325.0,Computers and Related products,Unfilled Orders
3,ADVM3UOCRPSAUS,2002-12-01,5774.0,Computers and Related products,Unfilled Orders
4,ADVM3UOCRPSAUS,2003-01-01,5508.0,Computers and Related products,Unfilled Orders


In [25]:
advm.to_parquet("data\\activity\\data_census_advm3.parquet")

## Advance Monthly Retail Sales (MARTS)

### Category Code

- `44X72` Retail Trade and Food Services
- `44Y72` Retail Trade and Food Services, ex Auto
- `44Z72` Retail Trade and Food Services, ex Gas
- `44W72` Retail Trade and Food Services, ex Auto and Gas
- `44000` Retail Trade
- `441` Motor Vehicle and Parts Dealers
- `441X` (4411,4412) Auto and Other Motor Vehicles
- `442` Furniture and Home Furnishings Stores
- `443` Electronics and Appliance Stores
- `444` Building Mat. and Garden Equip. and Supplies Dealers
- `445` Food and Beverage Stores
- `4451` Grocery Stores
- `446` Health and Personal Care Stores
- `447` Gasoline Stations
- `448` Clothing and Clothing Access. Stores
- `451` Sporting Goods, Hobby, Musical Instrument, and Book Stores
- `452` General Merchandise Stores
- `4521E` Department Stores
- `453` Miscellaneous Store Retailers
- `454` Nonstore Retailers
- `722` Food Services and Drinking Places

### Data Type

- `SM` Sales Monthly
- `MPCSM` Sales Monthly Percent Change

In [26]:
indicator = 'marts'
param = 'cell_value,time_slot_id'
cat = '&category_code' # all categories
datatype = '&data_type_code' # all data types
time = '&time=from+1990' # since start
geo = '&for=us'
adj = '&seasonally_adj=yes'


url = f'https://api.census.gov/data/timeseries/eits/{indicator}?get={param}{cat}{datatype}{geo}{adj}{time}&key={key}'

In [27]:
r = requests.get(url).json()
print(r[0])
print(r[1])

['cell_value', 'time_slot_id', 'category_code', 'data_type_code', 'seasonally_adj', 'time', 'us']
['1.4', '784', '445', 'E_SM', 'yes', '2015-04', '1']


In [28]:
retail = pd.DataFrame(data=r[1:], columns = r[0])
retail.info()
retail.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25597 entries, 0 to 25596
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cell_value      25597 non-null  object
 1   time_slot_id    25597 non-null  object
 2   category_code   25597 non-null  object
 3   data_type_code  25597 non-null  object
 4   seasonally_adj  25597 non-null  object
 5   time            25597 non-null  object
 6   us              25597 non-null  object
dtypes: object(7)
memory usage: 1.4+ MB


,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us
0,1.4,784,445,E_SM,yes,2015-04,1
1,2.1,783,453,E_SM,yes,2015-03,1
2,0.6,782,444,E_MPCSM,yes,2015-02,1
3,0.7,781,444,E_MPCSM,yes,2015-01,1
4,0.5,780,451,E_MPCSM,yes,2014-12,1


In [29]:
# changing columns dtypes:
retail = retail[retail["data_type_code"] == "SM"] #keep only Value of monthly sales 
retail["time"]= pd.to_datetime(retail["time"]+"-01")
retail["cell_value"]= pd.to_numeric(retail["cell_value"].str.replace(",", "", regex=False))

# creating series id
retail["series_id"] = "MARTS" + retail["data_type_code"] + retail["category_code"] + "SAUS"
retail.head()

,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us,series_id
19,209502,643,44W72,SM,yes,2003-07-01,1,MARTSSM44W72SAUS
20,291167,638,44X72,SM,yes,2003-02-01,1,MARTSSM44X72SAUS
21,305933,645,44X72,SM,yes,2003-09-01,1,MARTSSM44X72SAUS
22,227697,639,44Y72,SM,yes,2003-03-01,1,MARTSSM44Y72SAUS
23,234992,646,44Y72,SM,yes,2003-10-01,1,MARTSSM44Y72SAUS


Building Dictionary

In [30]:
category_dict = pd.DataFrame(
    [["44X72","Retail Trade and Food Services"],
     ["44Y72","Retail Trade and Food Services, ex Auto"],
     ["44Z72","Retail Trade and Food Services, ex Gas"],
     ["44W72","Retail Trade and Food Services, ex Auto and Gas"],
     ["44000","Retail Trade"],
     ["441","Motor Vehicle and Parts Dealers"],
     ["441X", "Auto and Other Motor Vehicles"],
     ["442","Furniture and Home Furnishings Stores"],
     ["443","Electronics and Appliance Stores"],
     ["444","Building Mat. and Garden Equip. and Supplies Dealers"],
     ["445","Food and Beverage Stores"],
     ["4451","Grocery Stores"],
     ["446","Health and Personal Care Stores"],
     ["447","Gasoline Stations"],
     ["448","Clothing and Clothing Access. Stores"],
     ["451","Sporting Goods, Hobby, Musical Instrument, and Book Stores"],
     ["452","General Merchandise Stores"],
     ["4521E","Department Stores"],["453","Miscellaneous Store Retailers"],
     ["454","Nonstore Retailers"],["722","Food Services and Drinking Places"]],
    columns = ["category_code","category_name"])

data_type_dict = pd.DataFrame(
    [["SM","Sales Monthly"]],
    columns = ["data_type_code","data_type_name"])

# merging with data:
retail = pd.merge(retail,data_type_dict,how="left",on="data_type_code")
retail = pd.merge(retail,category_dict,how="left",on="category_code")

In [31]:
# dropping and renaming columns:
retail = (
    retail[["series_id","time","cell_value","category_name","data_type_name"]]
    .rename(columns = {"cell_value":"value", "time":"date"})
    .sort_values(["category_name","date"])
)

retail.head()

,series_id,date,value,category_name,data_type_name
2126,MARTSSM441XSAUS,1992-01-01,30167,Auto and Other Motor Vehicles,Sales Monthly
2167,MARTSSM441XSAUS,1992-02-01,30457,Auto and Other Motor Vehicles,Sales Monthly
2153,MARTSSM441XSAUS,1992-03-01,29891,Auto and Other Motor Vehicles,Sales Monthly
2152,MARTSSM441XSAUS,1992-04-01,30361,Auto and Other Motor Vehicles,Sales Monthly
2213,MARTSSM441XSAUS,1992-05-01,30847,Auto and Other Motor Vehicles,Sales Monthly


Saving file:

In [32]:
retail.to_parquet("data\\activity\\data_census_retail.parquet")

## New Residential Sales (NRS)

### Category Code

- `SOLD` New Single-family Houses Sold
- `ASOLD` Annual Rate for New Single-family Houses Sold
- `FORSALE` New Single-family Houses For Sale

### Data Type

- `TOTAL` All Houses
- `NOTSTD` Houses that are Not Started
- `UNDERC` Houses that are Under Construction
- `COMPED` Houses that are Completed
- `MEDIAN` Median Sales Price
- `AVERAG` Average Sales Price
- `MONSUP` Months' Supply at Current Sales Rate
- `MMTHS` Median Number of Months For Sale Since Completion

### Error Type

- `E_TOTAL` Relative Standard Error for All Houses
- `E_NOTSTD` Relative Standard Error for Houses that are Not Started
- `E_UNDERC` Relative Standard Error for Houses that are Under Construction
- `E_COMPED` Relative Standard Error for Houses that are Completed
- `E_MEDIAN` Relative Standard Error for Median Sales Price
- `E_AVERAG` Relative Standard Error for Average Sales Price
- `E_MONSUP` Relative Standard Error for Months' Supply at Current Sales Rate
- `E_MMTHS` Relative Standard Error for Median Number of Months For Sale Since Completion



In [33]:
indicator = 'ressales'
param = 'cell_value,time_slot_id'
cat = '&category_code' # all categories
datatype = '&data_type_code' # all data types
time = '&time=from+1990' # since start
geo = '&for=us'
adj = '&seasonally_adj=yes'


url = f'https://api.census.gov/data/timeseries/eits/{indicator}?get={param}{cat}{datatype}{geo}{adj}{time}&key={key}'

In [34]:
r = requests.get(url).json()
print(r[0])
print(r[1])

['cell_value', 'time_slot_id', 'category_code', 'data_type_code', 'seasonally_adj', 'time', 'us']
['38', '712', 'FORSALE', 'NOTSTD', 'yes', '2009-04', '1']


In [35]:
nhs = pd.DataFrame(data=r[1:], columns = r[0])
nhs.info()
nhs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cell_value      3172 non-null   object
 1   time_slot_id    3172 non-null   object
 2   category_code   3172 non-null   object
 3   data_type_code  3172 non-null   object
 4   seasonally_adj  3172 non-null   object
 5   time            3172 non-null   object
 6   us              3172 non-null   object
dtypes: object(7)
memory usage: 173.6+ KB


,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us
0,38,712,FORSALE,NOTSTD,yes,2009-04,1
1,29,719,FORSALE,NOTSTD,yes,2009-11,1
2,85,630,FORSALE,COMPED,yes,2002-06,1
3,464,664,ASOLD,UNDERC,yes,2005-04,1
4,437,671,ASOLD,UNDERC,yes,2005-11,1


In [36]:
# changing columns dtypes:
nhs["time"]= pd.to_datetime(nhs["time"]+"-01")
nhs["cell_value"]= pd.to_numeric(nhs["cell_value"].str.replace(",", "", regex=False))

# creating series id
nhs["series_id"] = "RESSALES" + nhs["data_type_code"] + nhs["category_code"] + "SAUS"
nhs.head()

,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us,series_id
0,38.0,712,FORSALE,NOTSTD,yes,2009-04-01,1,RESSALESNOTSTDFORSALESAUS
1,29.0,719,FORSALE,NOTSTD,yes,2009-11-01,1,RESSALESNOTSTDFORSALESAUS
2,85.0,630,FORSALE,COMPED,yes,2002-06-01,1,RESSALESCOMPEDFORSALESAUS
3,464.0,664,ASOLD,UNDERC,yes,2005-04-01,1,RESSALESUNDERCASOLDSAUS
4,437.0,671,ASOLD,UNDERC,yes,2005-11-01,1,RESSALESUNDERCASOLDSAUS


In [37]:
category_dict = pd.DataFrame(
    [["SOLD","New Single-family Houses Sold"],
     ["ASOLD","Annual Rate for New Single-family Houses Sold"],
     ["FORSALE","New Single-family Houses For Sale"]],
    columns = ["category_code","category_name"])

data_type_dict = pd.DataFrame(
    [["TOTAL","All Houses"],
     ["NOTSTD","Houses that are Not Started"],
     ["UNDERC","Houses that are Under Construction"],
     ["COMPED","Houses that are Completed"],
     ["MEDIAN","Median Sales Price"],
     ["AVERAG","Average Sales Price"],
     ["MONSUP","Months' Supply at Current Sales Rate"],
     ["MMTHS","Median Number of Months For Sale Since Completion"]],
    columns = ["data_type_code","data_type_name"])

# merging with data:
nhs = pd.merge(nhs,data_type_dict,how="left",on="data_type_code")
nhs = pd.merge(nhs,category_dict,how="left",on="category_code")

In [38]:
# dropping and renaming columns:
nhs = (
    nhs[["series_id","time","cell_value","category_name","data_type_name"]]
    .dropna()
    .rename(columns = {"cell_value":"value", "time":"date"})
    .sort_values(["category_name","date"])
)

nhs.head()

,series_id,date,value,category_name,data_type_name
546,RESSALESTOTALASOLDSAUS,1990-01-01,620.0,Annual Rate for New Single-family Houses Sold,All Houses
543,RESSALESTOTALASOLDSAUS,1990-02-01,591.0,Annual Rate for New Single-family Houses Sold,All Houses
720,RESSALESTOTALASOLDSAUS,1990-03-01,574.0,Annual Rate for New Single-family Houses Sold,All Houses
719,RESSALESTOTALASOLDSAUS,1990-04-01,542.0,Annual Rate for New Single-family Houses Sold,All Houses
709,RESSALESTOTALASOLDSAUS,1990-05-01,534.0,Annual Rate for New Single-family Houses Sold,All Houses


In [39]:
# saving
nhs.to_parquet("data\\housing\\data_census_nhs.parquet")

## New Residential Construction (NRC)

### Category Code

- `APERMITS` Annual Rate for Housing Units Authorized in Permit-Issuing Places
- `PERMITS` Housing Units Authorized in Permit-Issuing Places
- `AUTHNOTSTD` Housing Units Authorized But Not Started
- `ASTARTS` Annual Rate for Housing Units Started
- `STARTS` Housing Units Started
- `UNDERCONST` Housing Units Under Construction
- `ACOMPLETIONS` Annual Rate for Housing Units Completed
- `COMPLETIONS` Housing Units Completed

### Data Type

- `TOTAL` Total Units
- `SINGLE` Single-family Units
- `MULTI` Units in Buildings with 5 Units or More

### Error Type

- `E_TOTAL` Relative Standard Error for Total Units
- `E_SINGLE` Relative Standard Error for Single-family Units
- `E_MULTI` Relative Standard Error for Units in Buildings with 5 Units or More


In [40]:
indicator = 'resconst'
param = 'cell_value,time_slot_id'
cat = '&category_code' # all categories
datatype = '&data_type_code' # all data types
time = '&time=from+1990' # since start
geo = '&for=us'
adj = '&seasonally_adj=yes'


url = f'https://api.census.gov/data/timeseries/eits/{indicator}?get={param}{cat}{datatype}{geo}{adj}{time}&key={key}'

In [41]:
r = requests.get(url).json()
print(r[0])
print(r[1])

['cell_value', 'time_slot_id', 'category_code', 'data_type_code', 'seasonally_adj', 'time', 'us']
['969', '0', 'APERMITS', 'TOTAL', 'yes', '2022-11', '1']


In [42]:
constr = pd.DataFrame(data=r[1:], columns = r[0])
constr.info()
constr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10883 entries, 0 to 10882
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cell_value      10883 non-null  object
 1   time_slot_id    10883 non-null  object
 2   category_code   10883 non-null  object
 3   data_type_code  10883 non-null  object
 4   seasonally_adj  10883 non-null  object
 5   time            10883 non-null  object
 6   us              10883 non-null  object
dtypes: object(7)
memory usage: 595.3+ KB


,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us
0,969,0,APERMITS,TOTAL,yes,2022-11,1
1,1460,0,ASTARTS,TOTAL,yes,2022-11,1
2,1503,0,ASTARTS,TOTAL,yes,2022-11,1
3,1109,0,ASTARTS,TOTAL,yes,2022-11,1
4,1289,0,ASTARTS,TOTAL,yes,2022-11,1


In [43]:
# changing columns dtypes:
constr["time"]= pd.to_datetime(constr["time"]+"-01")
constr["cell_value"]= pd.to_numeric(constr["cell_value"].str.replace(",", "", regex=False))

# creating series id
constr["series_id"] = "RESCONST" + constr["data_type_code"] + constr["category_code"] + "SAUS"
constr.head()

,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us,series_id
0,969,0,APERMITS,TOTAL,yes,2022-11-01,1,RESCONSTTOTALAPERMITSSAUS
1,1460,0,ASTARTS,TOTAL,yes,2022-11-01,1,RESCONSTTOTALASTARTSSAUS
2,1503,0,ASTARTS,TOTAL,yes,2022-11-01,1,RESCONSTTOTALASTARTSSAUS
3,1109,0,ASTARTS,TOTAL,yes,2022-11-01,1,RESCONSTTOTALASTARTSSAUS
4,1289,0,ASTARTS,TOTAL,yes,2022-11-01,1,RESCONSTTOTALASTARTSSAUS


In [44]:
category_dict = pd.DataFrame(
    [["APERMITS","Annual Rate for Housing Units Authorized in Permit-Issuing Places"],
     ["PERMITS","Housing Units Authorized in Permit-Issuing Places"],
     ["AUTHNOTSTD","Housing Units Authorized But Not Started"],
     ["ASTARTS","Annual Rate for Housing Units Started"],
     ["STARTS","Housing Units Started"],
     ["UNDERCONST","Housing Units Under Construction"],
     ["ACOMPLETIONS","Annual Rate for Housing Units Completed"],
     ["COMPLETIONS","Housing Units Completed"]],
    columns = ["category_code","category_name"])


data_type_dict = pd.DataFrame(
    [["TOTAL","Total Units"],
     ["SINGLE","Single-family Units"],
     ["MULTI","Units in Buildings with 5 Units or More"]],
    columns = ["data_type_code","data_type_name"])

# merging with data:
constr = pd.merge(constr,data_type_dict,how="left",on="data_type_code")
constr = pd.merge(constr,category_dict,how="left",on="category_code")

In [45]:
# dropping and renaming columns:
constr = (
    constr[["series_id","time","cell_value","category_name","data_type_name"]]
    .dropna()
    .rename(columns = {"cell_value":"value", "time":"date"})
    .sort_values(["category_name","date"])
)

constr.head()

,series_id,date,value,category_name,data_type_name
0,RESCONSTTOTALAPERMITSSAUS,2022-11-01,969,Annual Rate for Housing Units Authorized in Pe...,Total Units
30,RESCONSTTOTALAPERMITSSAUS,2022-11-01,961,Annual Rate for Housing Units Authorized in Pe...,Total Units
31,RESCONSTTOTALAPERMITSSAUS,2022-11-01,1000,Annual Rate for Housing Units Authorized in Pe...,Total Units
32,RESCONSTTOTALAPERMITSSAUS,2022-11-01,1002,Annual Rate for Housing Units Authorized in Pe...,Total Units
33,RESCONSTTOTALAPERMITSSAUS,2022-11-01,1027,Annual Rate for Housing Units Authorized in Pe...,Total Units


In [46]:
# saving
constr.to_parquet("data\\housing\\data_census_housing_starts.parquet")

## International Trade (FTD)

### Category Code

- `BOPGS` Balance of Payment Goods and Services
- `BOPG` Balance of Payment Goods

### Data Type

- `BAL` Balance
- `EXP` Exports
- `IMP` Imports

In [5]:
indicator = 'ftd'
param = 'cell_value,time_slot_id'
cat = '&category_code' # all categories
datatype = '&data_type_code' # all data types
time = '&time=from+1990' # since start
geo = '&for=us'
adj = '&seasonally_adj=yes'


url = f'https://api.census.gov/data/timeseries/eits/{indicator}?get={param}{cat}{datatype}{geo}{adj}{time}&key={key}'

In [6]:
r = requests.get(url).json()
print(r[0])
print(r[1])

['cell_value', 'time_slot_id', 'category_code', 'data_type_code', 'seasonally_adj', 'time', 'us']
['-5719', '528', 'BOPGS', 'BAL', 'yes', '1993-12', '1']


In [10]:
ftd = pd.DataFrame(data=r[1:], columns = r[0])
ftd.info()
ftd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cell_value      1107 non-null   object
 1   time_slot_id    1107 non-null   object
 2   category_code   1107 non-null   object
 3   data_type_code  1107 non-null   object
 4   seasonally_adj  1107 non-null   object
 5   time            1107 non-null   object
 6   us              1107 non-null   object
dtypes: object(7)
memory usage: 60.7+ KB


,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us
0,-5719,528,BOPGS,BAL,yes,1993-12,1
1,55552,528,BOPGS,EXP,yes,1993-12,1
2,61271,528,BOPGS,IMP,yes,1993-12,1
3,-7016,527,BOPGS,BAL,yes,1993-11,1
4,54345,527,BOPGS,EXP,yes,1993-11,1


In [11]:
# changing columns dtypes:
ftd["time"]= pd.to_datetime(ftd["time"]+"-01")
ftd["cell_value"]= pd.to_numeric(ftd["cell_value"].str.replace(",", "", regex=False))

# creating series id
ftd["series_id"] = "FTD" + ftd["data_type_code"] + ftd["category_code"] + "SAUS"
ftd.head()

,cell_value,time_slot_id,category_code,data_type_code,seasonally_adj,time,us,series_id
0,-5719,528,BOPGS,BAL,yes,1993-12-01,1,FTDBALBOPGSSAUS
1,55552,528,BOPGS,EXP,yes,1993-12-01,1,FTDEXPBOPGSSAUS
2,61271,528,BOPGS,IMP,yes,1993-12-01,1,FTDIMPBOPGSSAUS
3,-7016,527,BOPGS,BAL,yes,1993-11-01,1,FTDBALBOPGSSAUS
4,54345,527,BOPGS,EXP,yes,1993-11-01,1,FTDEXPBOPGSSAUS


In [12]:
category_dict = pd.DataFrame(
    [["BOPGS","Balance of Payment Goods and Services"],
     ["BOPG","Balance of Payment Goods"]],
    columns = ["category_code","category_name"])

data_type_dict = pd.DataFrame(
    [["BAL","Balance"],
     ["EXP","Exports"],
     ["IMP","Imports"]],
    columns = ["data_type_code","data_type_name"])

# merging with data:
ftd = pd.merge(ftd,data_type_dict,how="left",on="data_type_code")
ftd = pd.merge(ftd,category_dict,how="left",on="category_code")

In [14]:
# dropping and renaming columns:
ftd = (
    ftd[["series_id","time","cell_value","category_name","data_type_name"]]
    .dropna()
    .rename(columns = {"cell_value":"value", "time":"date"})
    .sort_values(["category_name","date"])
)

ftd.head()

,series_id,date,value,category_name,data_type_name
63,FTDEXPBOPGSSAUS,1992-01-01,50251,Balance of Payment Goods and Services,Exports
64,FTDIMPBOPGSSAUS,1992-01-01,52277,Balance of Payment Goods and Services,Imports
65,FTDBALBOPGSSAUS,1992-01-01,-2026,Balance of Payment Goods and Services,Balance
62,FTDBALBOPGSSAUS,1992-02-01,-831,Balance of Payment Goods and Services,Balance
69,FTDEXPBOPGSSAUS,1992-02-01,51682,Balance of Payment Goods and Services,Exports


In [16]:
# saving
ftd.to_parquet("data\\fiscal_trade\\data_census_ftd.parquet")